### Lath Essoh

# 2.Feature selection

In [2]:
import sys
sys.path.append("..") 
  
import pandas as pd
from itertools import combinations
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import math

# Objective: forecast the insurance charges for each customer
# The data contain some personal information for each individual

# # Immport the dataset in the working environment
dat = pd.read_csv('Insurance.csv')

# Overview of variables
dat.info()

# Convert categorical variables to dummy variables
dat = pd.get_dummies(dat, columns=['sex', 'smoker', 'region'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
# ********************************* Least Squares **********************************************

# Increase the number of predictors by interacting the original features 
from sklearn.preprocessing import PolynomialFeatures

# Increase the number of predictors by interacting the original features 
X = dat.drop(['charges'], axis = 1)
interaction = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X = pd.DataFrame(interaction.fit_transform(X), columns=interaction.get_feature_names_out())

# *** QUESTION: How many features does input X contain?

In [6]:
# Nombre de caractéristiques dans X
nombre_de_caracteristiques = X.shape[1]
print(f"Le nombre de caractéristiques dans X est : {nombre_de_caracteristiques}")

Le nombre de caractéristiques dans X est : 66


In [8]:
# Target
y = dat['charges']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Fit the model
lr = LinearRegression().fit(X_train, y_train)

# Make predictions on the test set
y_pred = lr.predict(X_test)

# Check the performance of the model
print('RMSE (test): %.2f' % math.sqrt(mean_squared_error(y_test, y_pred)))

RMSE (test): 4351.80


# QUESTION: Why is it more convenient to use RMSE instead of MSE in this context? 

L'utilisation de la RMSE (Root Mean Squared Error) plutôt que de la MSE (Mean Squared Error) est généralement plus pratique dans le contexte de l'évaluation des performances des modèles de régression pour plusieurs raisons :

Interprétation en Unités Réelles : La RMSE est exprimée dans les mêmes unités que la variable cible (dans ce cas, les charges d'assurance), ce qui facilite l'interprétation. Par exemple, si la RMSE est de 4351,80, cela signifie que, en moyenne, les prédictions du modèle s'écartent des valeurs réelles d'environ 4351,80 unités. En revanche, la MSE est exprimée en unités au carré, ce qui est moins intuitif.

Échelle des Erreurs : La RMSE est plus sensible aux grandes erreurs que la MSE car elle prend la racine carrée de la moyenne des carrés des erreurs. Cela donne plus de poids aux grandes erreurs, ce qui peut être important pour les applications où les grandes erreurs sont particulièrement problématiques.

Comparaison avec les Valeurs Observées : La RMSE permet de comparer directement la performance du modèle avec les valeurs observées. Par exemple, si l'on a des charges d'assurance typiques dans une certaine gamme, la RMSE donne une idée directe de l'écart moyen des prédictions par rapport à ces charges. La MSE, en revanche, donne un chiffre plus abstrait qui est moins directement comparable.

Sensibilité aux Erreurs : Puisque la RMSE est dans la même unité que la variable cible, elle est souvent plus facilement compréhensible pour les parties prenantes non techniques qui peuvent ne pas être familières avec les concepts statistiques mais peuvent comprendre les écarts en unités réelles.

En résumé, la RMSE est souvent préférée en raison de sa facilité d'interprétation et de sa capacité à fournir une mesure plus tangible de l'erreur moyenne des prédictions.

In [ ]:
# ********************************** Best-subset selection *************************************
# **********************************************************************************************

# scikit-learn does not provide built-in functions for this method, but we can use some of its tools to implement it

# Generate all possible feature subsets
def best_subset_selection(X_train, y_train, X_test, y_test):
    best_rmse_train = float('inf')
    best_rmse_test = float('inf')
    best_features = None
    
    features = X_train.columns
    for i in range(1, len(features) + 1):
        for subset in combinations(features, i):
            rmse_train, rmse_test = evaluate_subset(subset, X_train, y_train, X_test, y_test)
            if rmse_test < best_rmse_test:
                best_rmse_train, best_rmse_test = rmse_train, rmse_test
                best_features = subset
    
    return best_features, best_rmse_train, best_rmse_test

# Define the function to calculate performance of a subset of features
def evaluate_subset(features, X_train, y_train, X_test, y_test):
    model = LinearRegression()
    model.fit(X_train[list(features)], y_train)
    y_pred_train = model.predict(X_train[list(features)])
    y_pred_test = model.predict(X_test[list(features)])
    return mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test)

# Fit the models
best_features, best_rmse_train, best_rmse_test = best_subset_selection(X_train, y_train, X_test, y_test)

#...You can stop after a while because it will never converge!

# QUESTION: How many differnet models should the function estimate?

Pour déterminer combien de modèles différents la fonction `best_subset_selection` doit estimer, il est nécessaire de considérer le nombre de sous-ensembles de caractéristiques (features) que la fonction évalue. Puisque la fonction essaie toutes les combinaisons possibles des caractéristiques, le nombre total de modèles à estimer correspond au nombre de ces combinaisons.

In [10]:
# ********************************** Forward-stepwise selection **************************************

# Define the function for forward stepwise selection
def forward_stepwise_selection(X_train, y_train, X_test, y_test):
    remaining_features = list(X_train.columns)
    selected_features = []
    best_rmse_test = float('inf')

    while remaining_features:
        best_feature = None
        for feature in remaining_features:
            current_features = selected_features + [feature]
            rmse_train, rmse_test = evaluate_features(current_features, X_train, y_train, X_test, y_test)
            if rmse_test < best_rmse_test:
                best_rmse_test = rmse_test
                best_feature = feature
        
        if best_feature is None:
            break
        
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
    
    return selected_features, best_rmse_test

# Define the function to evaluate the performance of the model with the given features
def evaluate_features(features, X_train, y_train, X_test, y_test):
    model = LinearRegression()
    model.fit(X_train[features], y_train)
    y_pred_train = model.predict(X_train[features])
    y_pred_test = model.predict(X_test[features])
    return mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test)

# Fit the models
selected_features, best_rmse_test = forward_stepwise_selection(X_train, y_train, X_test, y_test)

# Check the selected features
print('Selected features:', selected_features)

# Check the performance of the model
print('RMSE (test): %.2f' % math.sqrt(best_rmse_test))

Selected features: ['bmi smoker_yes', 'age bmi', 'smoker_no', 'age', 'children', 'bmi region_northeast', 'sex_female region_northwest', 'age region_northeast', 'smoker_yes region_southeast', 'bmi sex_male', 'smoker_no region_southwest', 'age sex_female', 'bmi', 'age region_southwest', 'sex_female region_southwest']
RMSE (test): 4239.30


# QUESTIONS: 
#      - Can you conclude that this model performs better than least-squares regression?
#      - Does this model have a higher or lower bias than the least squares regression?
#      - Do you expect to find the same set of features if you used backward-stepwise selection?

### QUESTIONS

#### 1. Can you conclude that this model performs better than least-squares regression?

Oui, en se basant sur la comparaison des valeurs de la RMSE, on peut conclure que le modèle de sélection avant pas à pas (forward stepwise selection) est plus performant que le modèle de régression des moindres carrés. La RMSE pour le modèle de sélection avant pas à pas est de 4239,30, ce qui est inférieur à la RMSE de 4351,80 du modèle de régression des moindres carrés. Une RMSE plus faible signifie que les prédictions du modèle sont en moyenne plus proches des valeurs réelles.

#### 2. Does this model have a higher or lower bias than the least squares regression?

Ce modèle a probablement un biais plus élevé par rapport à la régression des moindres carrés. La sélection avant pas à pas ajoute les caractéristiques de manière incrémentale, ce qui peut simplifier le modèle en incluant seulement un sous-ensemble des caractéristiques. Cela peut introduire un biais plus élevé, tout en réduisant la variance. En revanche, le modèle de régression des moindres carrés utilise toutes les caractéristiques disponibles, ce qui entraîne un biais plus faible, mais une variance plus élevée. Comme le modèle de sélection avant pas à pas peut ne pas inclure toutes les caractéristiques, il est possible qu'il sous-ajuste légèrement les données, ce qui augmente le biais.


#### 3. Do you expect to find the same set of features if you used backward-stepwise selection?

Non, on ne s'attend pas à obtenir le même ensemble de caractéristiques avec la sélection arrière pas à pas. Les méthodes de sélection avant et arrière pas à pas conduisent souvent à des ensembles de caractéristiques différents car elles procèdent de manière opposée :

La sélection avant pas à pas commence avec aucune caractéristique et ajoute les plus significatives à chaque étape.
La sélection arrière pas à pas commence avec toutes les caractéristiques et retire les moins significatives à chaque étape.
Étant donné que ces méthodes suivent des chemins opposés, elles peuvent prioriser des caractéristiques différentes au cours du processus de sélection, ce qui conduit à des ensembles finaux de caractéristiques distincts.